In [ ]:
# Ставим библиотеки
!pip -q install transformers
!pip -q install bitsandbytes
!pip -q install accelerate
!pip -q install peft

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 73.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.6/251.6 kB 8.0 MB/s eta 0:00:00


In [ ]:
# Импорт библиотек
import torch
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split
import warnings
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
from tqdm import tqdm

nltk.download("punkt")
warnings.filterwarnings("ignore")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
# Скачиваем конфиг и настраиваем модель
# MODEL_NAME = "IlyaGusev/saiga_mistral_7b"

# config = PeftConfig.from_pretrained(MODEL_NAME)

# model = AutoModelForCausalLM.from_pretrained(
#     config.base_model_name_or_path,
#     load_in_8bit=True,
#     torch_dtype=torch.float16,
#     device_map="auto"
# )
# model = PeftModel.from_pretrained(
#     model,
#     MODEL_NAME,
#     torch_dtype=torch.float16
# )
# model.eval()

# tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=False)
# generation_config = GenerationConfig.from_pretrained(MODEL_NAME)

adapter_config.json:   0%|          | 0.00/480 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/623 [00:00<?, ?B/s]

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


pytorch_model.bin.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

...

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/90.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/96.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


generation_config.json:   0%|          | 0.00/265 [00:00<?, ?B/s]

In [ ]:
model_name = "IlyaGusev/saiga_mistral_7b_merged"
adapters_name = "ValterVar1/saiga-7b-lora-ner"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    load_in_4bit=True,
    torch_dtype=torch.float16
).eval()

model = PeftModel.from_pretrained(model, adapters_name)

### Prompt Engineering базовой модели Saiga Mistral 7B

In [ ]:
# Проверим работоспособность модели
text = "Расскажи, какие основные навыки нужны на IT позицию старшего продуктового аналитика данных"

final_text = ""
for message in [({"role": "user","content": text})]:
    message_text = "<s>{role}\n{content}</s>".format(**message)
    final_text += message_text
final_text += "<s>bot\n"
prompt = final_text.strip()

data = tokenizer(prompt, return_tensors="pt", add_special_tokens=False)
data = {k: v.to(model.device) for k, v in data.items()}
output_ids = model.generate(
    **data,
    generation_config=generation_config
)[0]
output_ids = output_ids[len(data["input_ids"][0]):]
output = tokenizer.decode(output_ids, skip_special_tokens=True)
result = output.strip()

print(result)

1. Программирование: Опыт работы с языками программирования, такими как Python или R, является обязательным для любого профессионала в области аналитики данных.
2. Математические навыки: Знание математических методов и алгоритмов, таких как статистический анализ, линейная регрессия, кластеризация и др., необходимо для успешного выполнения задач аналитика данных.
3. Базы данных: Опыт работы с базами данных, такими как SQL, NoSQL, и Hadoop, помогает создавать эффективные модели и анализировать большие объемы данных.
4. Аналитические инструменты: Знание различных аналитических инструментов, таких как Tableau, Power BI, QlikView, и Excel, позволяет создавать полезные графики и отчеты для анализа данных.
5. Коммуникативные навыки: Умение ясно и четко передавать свои результаты и мысли другим людям, включая технически неопытных коллег, является важным компонентом успешной работы аналитика данных.
6. Критический мышление: Умение критически оценивать данные и выявлять возможные ошибки или упущ

In [ ]:
# Проверяем работоспособность модели под нашу задачу через системный промпт
system_prompt = "Ты — Сайга, русскоязычный автоматический ассистент. Ты принимаешь на вход текст вакансий и должен выявлять и выводить основные навыки оттуда."
pre_text = "Распиши через запятую навыки из следующей вакансии: "
text = pre_text + "Обязанности:  Перевод технической документации Участие в совещаниях, перевод протоколов Ведение документооборота, реестров и отчетов Работа с копировально-множительной и архивной техникой  Требования:  Образование высшее Опыт работы от года Уверенный пользователь ПК (знание программы Excel) Свободное владение турецким языком (технический перевод)  Условия:  Заработная плата по результатам собеседования Премии по результатам работы Пятидневная рабочая неделя, полный рабочий день Добровольное медицинское страхование Оформление по ТК РФ Спорт и социальная программа для детей сотрудников"

messages = [{"role": "system", "content": system_prompt}]
messages.append({"role": "user","content": prompt})

final_text = ""
for message in messages:
    message_text = "<s>{role}\n{content}</s>".format(**message)
    final_text += message_text
final_text += "<s>bot\n"
prompt = final_text.strip()

data = tokenizer(prompt, return_tensors="pt", add_special_tokens=False)
data = {k: v.to(model.device) for k, v in data.items()}
output_ids = model.generate(
    **data,
    generation_config=generation_config
)[0]
output_ids = output_ids[len(data["input_ids"][0]):]
output = tokenizer.decode(output_ids, skip_special_tokens=True)
result = output.strip()

print(result)

1. Программирование: Опыт работы с языками программирования, такими как Python, R или Java, для создания моделей машинного обучения и анализа данных.
2. Машинное обучение: Знание различных методов машинного обучения, включая регрессию, классификацию, кросс-валидацию и оптимизацию моделей.
3. Анализ данных: Опыт работы с большими наборами данных, включая обработку, предобработку и визуализацию данных.
4. Базы данных: Знание баз данных, таких как SQL, NoSQL и Hadoop, для хранения и обработки данных.
5. Работа с облаком: Опыт работы с облачными платформами, такими как Amazon Web Services (AWS), Microsoft Azure или Google Cloud Platform (GCP).
6. Коммуникативные навыки: Умение эффективно коммуницировать свои результаты и предложения коллегам и руководству.
7. Использование инструментов: Знание инструментов для анализа данных, таких как Tableau, Power BI или QlikView.
8. Проектное управление: Опыт работы в рамках проектов, включая планирование, организацию и контроль за проектами.
9. Тестир

In [ ]:
# Попробуем Few-Shot Learning для лучшего понимания моделью нашей задачи
system_prompt = "Ты — Сайга, русскоязычный автоматический ассистент. Ты принимаешь на вход текст вакансий и должен выявлять и выводить основные навыки оттуда. Вот несколько примеров того, как ты должен работать:"
example_text_1 = "Текст вакансии: Обязанности:  Требуется водитель на Mercedes-Benz V Работа в сервисе Wheely, корпоративные трансферы и пр.  Требования:  Стаж от 3-х лет Хорошее знание Москвы  Условия:  Официальное трудоустройство по ТК График работы 1 2 или 2 2 38% от выручки ГСМ, мойка автомобиля, проезд по платным дорогам, парковка - за счет Работодателя."
example_output_1 = "Ожидаемый вывод модели: водительские навыки, знание города"
example_text_2 = "Текст вакансии: Обязанности: Работа с действующими клиентами. Обзвон потенциальных клиентов. Работа с документацией. Работа в Excel, crm. Требования: Знание ПК. Умение расположить клиента, позитивность, дружелюбность, клиентоориентированность, энергичность, высокая обучаемость, ответственность, трудолюбие. Условия: Оплата 1000 р. за смену. График работы 2 2. Трудоустройство по ТК."
example_output_2 = "Ожидаемый вывод модели: работа с клиентами, обзвон потенциальных клиентов, работа с документацией, работа в Excel, Excel, CRM"
example_text_3 = "Текст вакансии: Обязанности: Мойка посуды и других предметов с использованием специального оборудования и моющих средств Требования: - трудоустройство согласно ТК РФ;- график работы ( ежедневно, выходной- суббота)- предоставление форменной спецодежды  Условия: выходной 1 раз в неделю (суббота) рабочее место по адресу Большая 87д"
example_output_3 = "Ожидаемый вывод модели: мойка посуды, работа с оборудованием, использование моющих средств"
pre_text = "Распиши через запятую навыки из следующей вакансии: "
text = example_text_1 + "\n" + example_output_1 + "\n\n" + example_text_2 + "\n" + example_output_2 + "\n\n" + example_text_3 + "\n" + example_output_3 + "\n\n" + pre_text + "Обязанности:  Перевод технической документации Участие в совещаниях, перевод протоколов Ведение документооборота, реестров и отчетов Работа с копировально-множительной и архивной техникой  Требования:  Образование высшее Опыт работы от года Уверенный пользователь ПК (знание программы Excel) Свободное владение турецким языком (технический перевод)  Условия:  Заработная плата по результатам собеседования Премии по результатам работы Пятидневная рабочая неделя, полный рабочий день Добровольное медицинское страхование Оформление по ТК РФ Спорт и социальная программа для детей сотрудников"

messages = [{"role": "system", "content": system_prompt}]
messages.append({"role": "user","content": text})

final_text = ""
for message in messages:
    message_text = "<s>{role}\n{content}</s>".format(**message)
    final_text += message_text
final_text += "<s>bot\n"
prompt = final_text.strip()

data = tokenizer(prompt, return_tensors="pt", add_special_tokens=False)
data = {k: v.to(model.device) for k, v in data.items()}
output_ids = model.generate(
    **data,
    generation_config=generation_config
)[0]
output_ids = output_ids[len(data["input_ids"][0]):]
output = tokenizer.decode(output_ids, skip_special_tokens=True)
result = output.strip()

print(result)

Перевод технической документации, участие в совещаниях, перевод протоколов, ведение документооборота, реестры и отчеты, работа с копировально-множительной и архивными техниками, уверенный пользователь ПК (знание программы Microsoft Excel), свободное владение турецким языком (тематический перевод).


* Отлично, модель перестала пихать все подряд в навыки! Теперь для экономии пространства обернем preprocessing текста в функцию и оценим метрики качества на нескольких примерах.

In [ ]:
data = pd.read_json("data.json")
data

,id,vac_name,desc,values
0,10,Электромонтажник,Обязанности: - Обеспечивать бесперебойную и на...,"[{'skill': 'металлообработка', 'start': 121, '..."
1,11,Менеджер по продаже по недвижимости,Обязанности: Работа по заявкам для своих сотру...,"[{'skill': 'Уверенный пользователь ПК', 'start..."
2,12,Супервайзер мерчендайзеров,«OPEN Group» одна из крупнейших российских ком...,"[{'skill': 'Управление командой', 'start': 289..."
3,13,Продавец-консультант,"Ты наш человек, если понятия «честность», «отк...","[{'skill': 'честность', 'start': 30, 'end': 39..."
4,14,Мастер маникюра,"В связи с расширением штата, салон премиум кла...","[{'skill': 'доброжелательность', 'start': 135,..."
...,...,...,...,...
789,4191,JavaScript программист (игры),Ищем в команду программиста на проект гемблинг...,[{'skill': 'совместно с командой разрабатывать...
790,4192,Специалист по работе с клиентами,"Ты наш человек, если понятия «честность», «отк...","[{'skill': 'честность', 'start': 30, 'end': 39..."
791,4193,Руководитель департамента складской логистики,Обязанности: -Осуществляет руководство произво...,[{'skill': 'Осуществляет руководство производс...
792,4194,Охранник,"Работа в крупной, яркой компании BKC-Internati...",[{'skill': 'Охрана помещения и материальных це...


In [ ]:
def prompt_preprocessing(raw_text):

    system_prompt = "Ты — Сайга, русскоязычный автоматический ассистент. Ты принимаешь на вход текст вакансий и должен выявлять и выводить основные навыки оттуда в нижнем регистре на русском языке, разделяя их через знак ';'. Вот несколько примеров того, как ты должен работать:"
    example_text_1 = "Текст вакансии: Обязанности: - Обеспечивать бесперебойную и надежную работу основного технологического и вспомогательного оборудования - металлообработка; - Опыт работы в сборке и монтаже электрошкафов, прокладывание силовых кабелей и проводки, чтение электросхем. - Плановый и внеплановый ремонт электрооборудования. Требования: - Среднее профессиональное образование; - Опыт работы от 3 лет на аналогичной должности; - Знание технологического процесса ремонта, сборки, испытания и регулировки, правильность установки оборудования, агрегатов и машин. - Необходимо разбираться в механике, электрике, в пневматике. Условия: - График работы 5/2, - оформление по ТК РФ, - полная занятость."
    example_output_1 = "Ожидаемый вывод модели: металлообработка;сборке и монтаже электрошкафов;прокладывание силовых кабелей и проводки;чтение электросхем;ремонт электрооборудования;знание технологического процесса ремонта, сборки, испытания и регулировки;правильность установки оборудования, агрегатов и машин;необходимо разбираться в механике, электрике, в пневматике"
    example_text_2 = "Текст вакансии: Обязанности: Работа по заявкам для своих сотрудников. Ваши агенты работают на входящем клиентском трафике, который формируется за счет интернет. Образование не ниже среднего специального. Уверенный пользователь ПК. Презентабельный внешний вид. Умение обучать и направлять своих сотрудников на заключение сделок между клиентами и собственниками в покупке-продаже квартир. Ключевые навыки: навык аргументации и оказания влияния в продажах Условия: Премирование сотрудника ежемесячно. Сотрудникам без опыта работы бесплатное обучение. Рекламная, юридическая и информационная поддержка Полная занятость, гибкий график и возможность самостоятельно планировать свой рабочий день. Не имеющих опыта обучаем! Со всей комиссии расчет ЗП происходит в день сделки!!!!"
    example_output_2 = "Ожидаемый вывод модели: уверенный пользователь ПК;умение обучать;покупке-продаже квартир;навык аргументации;оказания влияния в продажах"
    example_text_3 = "Текст вакансии: Обязанности:  Выполнение плановых показателей Ведение и развитие клиентской базы Увеличение объемов продаж Работа с дебиторской задолженностью Ведение документооборота, отчетность  Требования:  Образование - средне-специальное, высшее Опыт работы мерчендайзером, торговым представителем - желателен Уверенный пользователь компьютера Наличие водительских прав, личного автомобиля Личностные качества: коммуникабельность, активность, ответственность  Условия:  Оформление в штат дистрибьютора, социальный пакет Компенсация ГСМ, сотовой связи"
    example_output_3 = "Ожидаемый вывод модели: ведение и развитие клиентской базы;работа с дебиторской задолженностью;ведение документооборота;отчетность;уверенный пользователь компьютера;коммуникабельность;активность;ответственность"
    pre_text = "По формату из примеров распиши навыки из следующей вакансии: "
    text = example_text_1 + "\n" + example_output_1 + "\n\n" + example_text_2 + "\n" + example_output_2 + "\n\n" + example_text_3 + "\n" + example_output_3 + "\n\n" + pre_text + raw_text

    messages = [{"role": "system", "content": system_prompt}]
    messages.append({"role": "user","content": text})

    final_text = ""
    for message in messages:
        message_text = "<s>{role}\n{content}</s>".format(**message)
        final_text += message_text
    final_text += "<s>bot\n"
    prompt = final_text.strip()

    data = tokenizer(prompt, return_tensors="pt", add_special_tokens=False)
    data = {k: v.to(model.device) for k, v in data.items()}
    output_ids = model.generate(
        **data,
        generation_config=generation_config
    )[0]
    output_ids = output_ids[len(data["input_ids"][0]):]
    output = tokenizer.decode(output_ids, skip_special_tokens=True)
    result = output.strip()

    return result

In [ ]:
def create_bio_tagged_data(id, vac_name, desc, values):

    tokenized_desc = word_tokenize(desc.lower(), language="russian")
    tokenized_vals = list(map(lambda x: word_tokenize(x["skill"].lower(), language="russian"), values))

    new_data = []
    i = 0
    while i < len(tokenized_desc):
        token = tokenized_desc[i]
        tag = 'O'

        for skill_tokens in tokenized_vals:
            if tokenized_desc[i:i+len(skill_tokens)] == skill_tokens:
                tag = 'B-SKILL'
                new_data.append((id, vac_name, token, tag))
                for j in range(1, len(skill_tokens)):
                    i += 1
                    token = tokenized_desc[i]
                    tag = 'I-SKILL'
                    new_data.append((id, vac_name, token, tag))
                break

        if tag == 'O':
            new_data.append((id, vac_name, token, tag))
        i += 1

    return pd.DataFrame(new_data, columns=['id', 'position', 'token', 'tag'])

def apply_to_row(row):
    return create_bio_tagged_data(row['id'], row['vac_name'], row['desc'], row['values'])

In [ ]:
def create_bio_tagged_output(id, vac_name, desc, model_output):

    tokenized_desc = word_tokenize(desc.lower(), language="russian")
    tokenized_vals = list(map(lambda x: word_tokenize(x.lower(), language="russian"), model_output.split(";")))

    new_data = []
    i = 0
    while i < len(tokenized_desc):
        token = tokenized_desc[i]
        tag = 'O'

        for skill_tokens in tokenized_vals:
            if tokenized_desc[i:i+len(skill_tokens)] == skill_tokens:
                tag = 'B-SKILL'
                new_data.append((id, vac_name, token, tag))
                for j in range(1, len(skill_tokens)):
                    i += 1
                    token = tokenized_desc[i]
                    tag = 'I-SKILL'
                    new_data.append((id, vac_name, token, tag))
                break

        if tag == 'O':
            new_data.append((id, vac_name, token, tag))
        i += 1

    return pd.DataFrame(new_data, columns=['id', 'position', 'token', 'tag'])

In [ ]:
def calculate_metrics(predictions, true_tags):

    precision = precision_score(true_tags, predictions, labels=["B-SKILL", "I-SKILL"], average='weighted')
    recall = recall_score(true_tags, predictions, labels=["B-SKILL", "I-SKILL"], average='weighted')
    f1 = f1_score(true_tags, predictions, labels=["B-SKILL", "I-SKILL"], average='weighted')

    return (precision, recall, f1)

In [ ]:
sample = data["id"].values
precision_lst = []
recall_lst = []
f1_lst = []
for id in sample:

    data_temp = data[data["id"] == id].copy()
    output = prompt_preprocessing(data_temp["desc"].values[0])

    compare = create_bio_tagged_data(data_temp["id"].values[0], data_temp["vac_name"].values[0], data_temp["desc"].values[0], data_temp["values"].values[0])
    compare_model = create_bio_tagged_output(data_temp["id"].values[0], data_temp["vac_name"].values[0], data_temp["desc"].values[0], output)

    precision, recall, f1 = calculate_metrics(compare_model["tag"], compare["tag"])
    precision_lst.append(precision)
    recall_lst.append(recall)
    f1_lst.append(f1)

print("Average Precision:", round(sum(precision_lst) / len(precision_lst), 4))
print("Average Recall:", round(sum(recall_lst) / len(recall_lst), 4))
print("Average F1:", round(sum(f1_lst) / len(f1_lst), 4))


Average Precision: 0.7378
Average Recall: 0.7335
Average F1: 0.7354


In [ ]:
print("Average Precision:", round(sum(precision_lst) / len(precision_lst), 4))
print("Average Recall:", round(sum(recall_lst) / len(recall_lst), 4))
print("Average F1:", round(sum(f1_lst) / len(f1_lst), 4))

Average Precision: 0.7378
Average Recall: 0.7335
Average F1: 0.7354
